In [1]:
#from barney_functions import add_labels_to_excel,gen_audio_array,buffer,shifter,louder,plot_mel,augment_audio,plot_sample,hsr_loader,lsr_loader,calc_stft,calc_melstft,spec_plot, gen_labels, gen_audio_array_noexcel,augment_audio_faster
from barney_functions import *

In [2]:
import librosa
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random
import re
import os
from tqdm import tqdm 
import soundfile as sf

# Load Zebra

In [3]:
audio_path = 'zebra audio sample_Bing_413/all/'
audio_path_long = 'Audio Files All/'
spreadsheet_path = 'Zebras.Assumption.data_Bing_413 .xlsx'
df = add_labels_to_excel(spreadsheet_path)
#audio_files, audio_size, index_longest = gen_audio_array(audio_path,df)
zebra_labels = df['label']

audio_files, index_longest, audio_size = hsr_loader(audio_path,df)
#zebra_labels = gen_labels(filenames_long)

Finding longest


100%|████████████████████████████████████████| 413/413 [00:00<00:00, 894.01it/s]


Loading files


100%|████████████████████████████████████████| 413/413 [00:00<00:00, 924.86it/s]


In [4]:
balanced, balanced_labels = balancer(audio_files,zebra_labels)

Initial class proportions:
[0.574 0.034 0.092 0.3  ]
----------------------
Balancing: snort
Multiplier: 1.0
----------------------
Balancing: softsnort
Multiplier: 11.0
----------------------
Balancing: squeal
Multiplier: 3.0
----------------------
Balancing: whinnie
Multiplier: 1.0
----------------------
Final class proportions:
[0.377 0.245 0.181 0.197]


# Generate not zebera

In [ ]:
nzebra_path = 'big bounce small.wav'
nzebra_audio, labels_nzebra = gen_nzebra(nzebra_path,audio_size)
#save_nzebra(nzebra_audio)

Padding


  0%|                                                  | 0/1599 [00:00<?, ?it/s]

In [ ]:
all_audio, all_lables = add_nzebra(audio_files,zebra_labels,nzebra_audio,labels_nzebra)

In [ ]:
print("Unaugmented dataset size: ",all_audio.nbytes/(1e9), 'GB')

# Augment Audio

In [ ]:
augmented_audio = augment_audio_faster_smaller(audio_files)

In [ ]:
print(augmented_audio.shape)
print("Augmented dataset size: ",augmented_audio.nbytes/(1e9), 'GB')

In [ ]:
plt.plot(audio_files[400])

In [ ]:
plot_sample(augmented_audio,400)

In [ ]:
stfts = calc_stft(augmented_audio)
#mels = calc_melstft(augmented_audio)

In [ ]:
print(stfts.shape)
print("Augmented dataset size: ",stfts.nbytes/(1e9), 'GB')

In [ ]:
spec_plot(stfts[2,400,:,:])

In [ ]:
stfts_1D = np.zeros(stfts[0].shape)
stfts_1D = stfts[0]
for i in tqdm(range(1,stfts.shape[0])):
    stfts_1D = np.concatenate((stfts_1D,stfts[i]), axis = 0)
stfts = None

In [ ]:
stfts_1D.shape
print("Dataset size: ",stfts_1D.nbytes/(1e9), 'GB')

In [ ]:
#np.savez_compressed('stfts_array',stfts_1D)
#stfts = np.load('stfts_array.npz')

In [ ]:
import librosa.display

In [ ]:
spec_plot(stfts_1D[0])

In [ ]:
listen(librosa.istft(stfts_1D[1220]))